In [18]:
from dotenv import load_dotenv
from pathlib import Path
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from typing import List, Tuple
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from transformers import Pipeline
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.docstore.document import Document as LangchainDocument

In [4]:
load_dotenv()

False

# load and chunk

In [5]:
pdf_files = Path("data").glob("*.pdf")
text = ""

for pdf_file in pdf_files:
    reader = PdfReader(pdf_file)
for page in reader.pages:
    text += page.extract_text() + "\n\n"

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)


In [6]:
# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=40,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
# for doc in text:
#     docs_processed += text_splitter.split_documents([doc])
#docs_processed += text_splitter.split_documents(text)
docs_processed += text_splitter.split_text(text)

In [7]:
len(docs_processed), type(docs_processed)

(96, list)

# embed and save the VDB for free

In [8]:
len(docs_processed), type(docs_processed)

(96, list)

In [ ]:
#!KERNEL CRASH
# EMBEDDING_MODEL_NAME = "thenlper/gte-small"
# embedding_model = HuggingFaceEmbeddings(
#     model_name=EMBEDDING_MODEL_NAME,
#     multi_process=True,
#     model_kwargs={"device": "cpu"}, #cuda
#     encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
# )

In [9]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [10]:
# Kernel test
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.038338541984558105, 0.1234646886587143, -0.028642907738685608]

In [11]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_texts(
    docs_processed, embeddings, distance_strategy=DistanceStrategy.COSINE
)

In [13]:
# quick querry test
query_text = "Does Antoine speak spanish?"
KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=query_text, k=1)

[Document(page_content='Name: Antoine Bertin\nAge: 35 years old\nOccupation: Data Scientist, Former Sommelier & Blockchain Enthusiast\nLink to schedule a call: [https://calendly.com/antoinebertin/30](https://calendly.com/antoinebertin/30)\nEmail: [antoinebe35gmail.com](mailto:antoinebe35gmail.com)\nLanguages:\n* French\n* Spanish\n* English\nAbout me:')]

In [22]:
# Save the VDB with langchain instance
KNOWLEDGE_VECTOR_DATABASE.save_local("open_source_model_vdb")

# Load the VDB

In [6]:
#! Friendly reminder -> cell to run if cell above not ran 
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
# Load the vdb
vdb = FAISS.load_local("open_source_model_vdb", embeddings, allow_dangerous_deserialization=True) #  take the same model used for the embeddings

In [8]:
# quick test
# quick querry test
query_text = "Does Antoine speak spanish?"
docs = vdb.similarity_search(query=query_text, k=1)
docs

[Document(page_content='Name: Antoine Bertin\nAge: 35 years old\nOccupation: Data Scientist, Former Sommelier & Blockchain Enthusiast\nLink to schedule a call: [https://calendly.com/antoinebertin/30](https://calendly.com/antoinebertin/30)\nEmail: [antoinebe35gmail.com](mailto:antoinebe35gmail.com)\nLanguages:\n* French\n* Spanish\n* English\nAbout me:')]

In [12]:
docs[0].page_content[:20]

'Name: Antoine Bertin'

# add LLM Reader

In [14]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context, give a comprehensive answer to the question. 
        Respond only to the question asked, response should be concise and relevant to the question. 
        Do not respond anything that can't be answered from the context. If the answer cannot be deduced from the contextcsuggest the following:
- Schedule a video call with Antoine Bertin for more detailed information. You can do this by visiting his Calendly link: 'https://calendly.com/antoinebertin/30'.
- Learn more about Antoine's story, you can download the 'antoine' package. To do this, use the command 'pip install antoine' in your terminal and run the 'hire_antoine' command.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]

In [15]:
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

In [16]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME)#, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

Loading checkpoint shards: 100%|██████████| 8/8 [01:22<00:00, 10.37s/it]


In [17]:
# check prompt template
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>
Using the information contained in the context, give a comprehensive answer to the question. 
        Respond only to the question asked, response should be concise and relevant to the question. 
        Do not respond anything that can't be answered from the context. If the answer cannot be deduced from the contextcsuggest the following:
- Schedule a video call with Antoine Bertin for more detailed information. You can do this by visiting his Calendly link: 'https://calendly.com/antoinebertin/30'.
- Learn more about Antoine's story, you can download the 'antoine' package. To do this, use the command 'pip install antoine' in your terminal and run the 'hire_antoine' command.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}</s>
<|assistant|>



In [41]:
# quick test
READER_LLM("What is 4+4? Answer:")

[{'generated_text': ' 8\n\nQuestion: How many sides does a triangle have? Answer: 3\n\nQuestion: Which planet is known as the Red Planet? Answer: Mars\n\nQuestion: Who painted the Mona Lisa? Answer: Leonardo da Vinci\n\nQuestion: Which country won the FIFA World Cup in 2018? Answer: France\n\nQuestion: Who wrote the novel "Pride and Prejudice"? Answer: Jane Austen\n\nQuestion: Which animal is found on the Australian coat of arms? Answer: Kangaroo\n\nQuestion: In which ocean is the Great Barrier Reef located? Answer: Coral Sea (which is part of the Pacific Ocean)\n\nQuestion: Which country hosted the 2016 Summer Olympics? Answer: Brazil\n\nQuestion: Which famous landmark is located in New York City? Answer: Statue of Liberty\n\nQuestion: Which continent is the largest in terms of area? Answer: Asia\n\nQuestion: Which country is known for its famous cherry blossoms? Answer: Japan\n\nQuestion: Which country has the longest coastline in the world? Answer: Canada\n\nQuestion: Which country 

In [ ]:
# pas de reranker sur un petit doc
# from ragatouille import RAGPretrainedModel
# RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
# TODO https://huggingface.co/learn/cookbook/en/advanced_rag

In [20]:
def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    num_retrieved_docs: int = 2,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [21]:
query_text = "Does Antoine speak spanish?"

answer, relevant_docs = answer_with_rag(query_text, READER_LLM, vdb)

=> Retrieving documents...
=> Generating answer...


In [22]:
answer

'Yes, according to the context provided, Antoine speaks Spanish as one of his languages.'

In [23]:
relevant_docs

['Name: Antoine Bertin\nAge: 35 years old\nOccupation: Data Scientist, Former Sommelier & Blockchain Enthusiast\nLink to schedule a call: [https://calendly.com/antoinebertin/30](https://calendly.com/antoinebertin/30)\nEmail: [antoinebe35gmail.com](mailto:antoinebe35gmail.com)\nLanguages:\n* French\n* Spanish\n* English\nAbout me:',
 "If you want to contribute to the 'antoine' package and make me better, check out my GitHub: [http://github.com/monolok/antoine](http://github.com/monolok/antoine)\nSkills:\n* Project Management\n* Business Development\n* Process Improvement\n* Process Automation\n* Cluster Analysis\n* System Deployment\n* Prediction\n* Deep Learning\n* Machine Learning\n* Big Data\n* Big Data Analytics\n* Data Mining"]